In [1]:
import torch

import header_code

torch._logging.set_logs(graph_breaks=True, graph_code=True)

# Disabling and Suppressing Errors
For some model architectures, there are portions of the model which are particularly difficult to compile -
either there are many graph breaks, or there are crashes.
You may want to explicitly disable these portions of the model which are problematic so that you can apply
`torch.compile` to the parts that work. You can do this by using the `@torch.compiler.disable` decorator.
When `torch.compile` attempts to call a disabled function, it breaks the graph and skips tracing the disabled function,
resuming tracing after the call. By default, all recursive calls made from a disabled function are also disabled.
Use the `recursive=False` option to allow compilation for recursive calls.

In [2]:
def inner1(x):
    torch._dynamo.graph_break()  # not traced
    return x + 1  # not traced

@torch.compiler.disable
def outer1(x):
    x = x + 2  # not traced
    torch._dynamo.graph_break()  # not traced
    return inner1(x)

@torch.compile
def f(x):
    x = outer1(x)
    return x + 4  # traced

print(f(torch.ones(3)))

Graph break in user code at C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\1421264493.py:13
Graph Break Reason: Skip calling `torch.compiler.disable()`d function
  Explanation: Skip calling function `<function outer1 at 0x000002C3749D84A0>` since it was wrapped with `torch.compiler.disable` (reason: None)
  Hint: Remove the `torch.compiler.disable` call

  Developer debug context: <function outer1 at 0x000002C3749D84A0>

 For more details about this graph break, please visit: https://meta-pytorch.github.io/compile-graph-break-site/gb/gb0098.html
User code traceback:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", l

TRACED GRAPH
 ===== __compiled_fn_4_80fd10a1_9ee4_4144_9391_bdc7cb4aac13 =====
 C:\dev\pytorch\venv\Lib\site-packages\torch\fx\_lazy_graph_module.py class GraphModule(torch.nn.Module):
    def forward(self, L_stack0_: "f32[3][1]cpu"):
        l_stack0_ = L_stack0_
        
         # File: C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\1421264493.py:14 in torch_dynamo_resume_in_f_at_13, code: return x + 4  # traced
        add: "f32[3][1]cpu" = l_stack0_ + 4;  l_stack0_ = None
        return (add,)
        



tensor([8., 8., 8.])


In [3]:
def inner2(x):
    torch._dynamo.graph_break()  # traced
    return x + 1  # traced

@torch.compiler.disable(recursive=False)
def outer2(x):
    x = x + 2  # not traced
    torch._dynamo.graph_break()  # not traced
    return inner2(x)

@torch.compile
def g(x):
    x = outer2(x)
    return x + 4  # traced

print(g(torch.ones(3)))

Graph break in user code at C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\881423632.py:13
Graph Break Reason: Skip inlining `torch.compiler.disable()`d function
  Explanation: Skip inlining function <function outer2 at 0x000002C3787AFE20> since it was wrapped with `torch.compiler.disable` (reason: None)
  Hint: Remove the `torch.compiler.disable` call

  Developer debug context: <function outer2 at 0x000002C3787AFE20>

 For more details about this graph break, please visit: https://meta-pytorch.github.io/compile-graph-break-site/gb/gb0099.html
User code traceback:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", li

Graph break in user code at C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\881423632.py:8
Graph Break Reason: Call to `torch._dynamo.graph_break()`
  Explanation: User-inserted graph break. Message: None
  Hint: Remove the `torch._dynamo.graph_break()` call.

  Developer debug context: Called `torch._dynamo.graph_break()` with args `[]`, kwargs `{}`

 For more details about this graph break, please visit: https://meta-pytorch.github.io/compile-graph-break-site/gb/gb0025.html
User code traceback:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\L

TRACED GRAPH
 ===== __compiled_fn_10_9b6aaceb_bfaf_416e_b3f9_fa8d5fe267ef =====
 C:\dev\pytorch\venv\Lib\site-packages\torch\fx\_lazy_graph_module.py class GraphModule(torch.nn.Module):
    def forward(self, L_x_: "f32[3][1]cpu"):
        l_x_ = L_x_
        
         # File: C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\881423632.py:7 in outer2, code: x = x + 2  # not traced
        x: "f32[3][1]cpu" = l_x_ + 2;  l_x_ = None
        return (x,)
        



Graph break in user code at C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\881423632.py:2
Graph Break Reason: Call to `torch._dynamo.graph_break()`
  Explanation: User-inserted graph break. Message: None
  Hint: Remove the `torch._dynamo.graph_break()` call.

  Developer debug context: Called `torch._dynamo.graph_break()` with args `[]`, kwargs `{}`

 For more details about this graph break, please visit: https://meta-pytorch.github.io/compile-graph-break-site/gb/gb0025.html
User code traceback:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\L

Graph break (user stack suppressed due to duplicate graph break) in user code at C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\881423632.py:2
Graph Break Reason: Call to `torch._dynamo.graph_break()`
  Explanation: User-inserted graph break. Message: None
  Hint: Remove the `torch._dynamo.graph_break()` call.

  Developer debug context: Called `torch._dynamo.graph_break()` with args `[]`, kwargs `{}`

 For more details about this graph break, please visit: https://meta-pytorch.github.io/compile-graph-break-site/gb/gb0025.html


TRACED GRAPH
 ===== __compiled_fn_18_21aa0ac2_858a_4b75_b9b5_7e3b9ba8bff7 =====
 C:\dev\pytorch\venv\Lib\site-packages\torch\fx\_lazy_graph_module.py class GraphModule(torch.nn.Module):
    def forward(self, L_x_: "f32[3][1]cpu"):
        l_x_ = L_x_
        
         # File: C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\881423632.py:3 in torch_dynamo_resume_in_inner2_at_2, code: return x + 1  # traced
        add: "f32[3][1]cpu" = l_x_ + 1;  l_x_ = None
        return (add,)
        



TRACED GRAPH
 ===== __compiled_fn_20_03fb39dd_b270_47f2_963f_61e39076f0e5 =====
 C:\dev\pytorch\venv\Lib\site-packages\torch\fx\_lazy_graph_module.py class GraphModule(torch.nn.Module):
    def forward(self, L_stack0_: "f32[3][1]cpu"):
        l_stack0_ = L_stack0_
        
         # File: C:\Users\Aditya\AppData\Local\Temp\ipykernel_6152\881423632.py:14 in torch_dynamo_resume_in_g_at_13, code: return x + 4  # traced
        add: "f32[3][1]cpu" = l_stack0_ + 4;  l_stack0_ = None
        return (add,)
        



tensor([8., 8., 8.])


For example, one can use `torch.compiler.disable` to disable `torch.compile` on sparse architecture in
recommendation models, as the sparse arch is difficult to compile.
Preprocessing and logging functions are other examples of functions that typically cause
a lot of graph breaks and do not get value from being compiled.

If you are experiencing compiler crashes and you want to continue regardless,
you can set `torch._dynamo.config.suppress_errors = True`.
When the compiler crashes, we will just skip tracing the function and try again later.
**This is not best practice** - it is better to eventually manually add `disable` annotations as necessary.